In [ ]:
import os
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
from src.utils.general import get_db_conn, get_boto3_session, load_model_s3, load_matrix_s3
from src.utils import project_constants as constants

In [ ]:
cred_file = '../conf/local/credentials.yaml'
db_conn=get_db_conn(cred_file)

##### Summary of data we received from Legiscan

In [ ]:
summary_file = '/mnt/data/projects/aclu_leg_tracker/legiscan_dump_20200615_processed/data_dump_summary.csv'

In [ ]:
dump_summary = pd.read_csv(summary_file)

##### Summary of what we have in the DB

In [ ]:
q = """
    select 
        session_id, 
        max(state) as state,
        max(session_title) as session_name,
        max(year_start) as year_start,
        max(year_end) as year_end,
        count(distinct bill_id) as num_bills, 
        count(distinct doc_id) as num_texts,
        count(distinct people_id) as num_people
    from clean.bills left join clean.sessions using(session_id) 
        left join clean.bill_docs using(bill_id)
            left join clean.session_people using(session_id)
    group by session_id
    order by state
"""

In [ ]:
db_summary = pd.read_sql(q, db_conn)

#### Comparing

In [ ]:
merged_df = dump_summary.merge(db_summary, on='session_id', suffixes=('_dump', '_db'), how='left')

**Full sessions that are missing**

In [ ]:
msk = merged_df.isna().any(axis=1)
print(len(merged_df[msk]))
merged_df[msk]

In [ ]:
len(merged_df[msk]['state_dump'].unique())

**Incomplete Sessions**

In [ ]:
merged_df['diff'] = merged_df['num_bills_dump'] - merged_df['num_bills_db']  
msk = (merged_df['diff'] > 0) & (merged_df['state_dump'] != 'US')
tmp = merged_df[msk].sort_values('diff', ascending=False)
tmp
# cols = ['state_dump', 'year_start', 'year_end', 'session_name_dump', 'num_texts_dump', 'num_texts_db', 'diff']


In [ ]:
state_missing_bills = tmp.groupby('state_dump').sum()['diff'].sort_values(ascending=False).reset_index()

In [ ]:
state_missing_bills

In [ ]:
if len(state_missing_bills) != 0: 
    fig, ax = plt.subplots(figsize=(14,5))
    sns.barplot(data=state_missing_bills, x='state_dump', y='diff', color='royalblue')

    ax.set_xlabel('State')
    ax.set_ylabel('No. Bills missing in the DB')
    sns.despine()

In [16]:
state_missing_bills.sum()

state_dump    TNWALAVTDCWIMDORIARIDEKSNVSDUTINFLWYTXPAAZILNJ...
diff                                                      76053
dtype: object

In [15]:
1309921 - 1204567

105354